<a href="https://colab.research.google.com/github/ibtiheljl/ibtiheljl/blob/main/SOC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import keras
import os
import glob
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
import math

import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
from keras.preprocessing.sequence import TimeseriesGenerator
from keras.models import Sequential
from keras.layers import Dense, LSTM, SimpleRNN, Dropout
from keras.optimizers import RMSprop
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.models import model_from_json
from keras import optimizers

1.Load Dataset

In [24]:
dir = r'/.ipynb_checkpoints'

# Check if the directory exists
if os.path.exists(dir):
    listdir = os.listdir(dir)
    print(listdir)
    print("The number of datasets:", len(listdir))
else:
    print(f"The directory '{dir}' does not exist.")

['B07_discharge_soh.csv', 'B48_discharge_soh.csv', 'tempCodeRunnerFile.python', '.ipynb_checkpoints', 'B33_discharge_soh.csv', 'B18_discharge_soh.csv', 'B47_discharge_soh.csv', 'B05_discharge_soh.csv', 'B46_discharge_soh.csv', 'B34_discharge_soh.csv']
The number of datasets: 10


In [33]:
import os
import pandas as pd
import numpy as np

num = ['B05', 'B07', 'B18', 'B33', 'B34', 'B46', 'B47', 'B48']

for i in range(len(num)):
    vector = np.zeros((1, 3))
    # Corrected path construction
    path = os.path.join(os.getcwd(), '.ipynb_checkpoints', num[i] + '_discharge_soh.csv')

    try:
        csv = pd.read_csv(path)
        df = pd.DataFrame(csv)

        vec = df[['cycle', 'capacity', 'SOH']]

        globals()['data_{}'.format(num[i])] = vec
    except FileNotFoundError:
        print(f"File not found: {path}")


File not found: /content/.ipynb_checkpoints/B05_discharge_soh.csv
File not found: /content/.ipynb_checkpoints/B07_discharge_soh.csv
File not found: /content/.ipynb_checkpoints/B18_discharge_soh.csv
File not found: /content/.ipynb_checkpoints/B33_discharge_soh.csv
File not found: /content/.ipynb_checkpoints/B34_discharge_soh.csv
File not found: /content/.ipynb_checkpoints/B46_discharge_soh.csv
File not found: /content/.ipynb_checkpoints/B47_discharge_soh.csv
File not found: /content/.ipynb_checkpoints/B48_discharge_soh.csv


In [30]:
data = pd.read_csv('/.ipynb_checkpoints/B05_discharge_soh.csv')
df = pd.DataFrame(data)
df

,terminal_voltage,terminal_current,temperature,charge_current,charge_voltage,time,capacity,cycle,SOH
0,4.191492,-0.004902,24.330034,-0.0006,0.000,0.000,1.856487,1,0.928244
1,4.190749,-0.001478,24.325993,-0.0006,4.206,16.781,1.856487,1,0.928244
2,3.974871,-2.012528,24.389085,-1.9982,3.062,35.703,1.856487,1,0.928244
3,3.951717,-2.013979,24.544752,-1.9982,3.030,53.781,1.856487,1,0.928244
4,3.934352,-2.011144,24.731385,-1.9982,3.011,71.922,1.856487,1,0.928244
...,...,...,...,...,...,...,...,...,...
50280,3.579262,-0.001569,34.864823,0.0006,0.000,2781.312,1.325079,168,0.662540
50281,3.581964,-0.003067,34.814770,0.0006,0.000,2791.062,1.325079,168,0.662540
50282,3.584484,-0.003079,34.676258,0.0006,0.000,2800.828,1.325079,168,0.662540
50283,3.587336,0.001219,34.565580,0.0006,0.000,2810.640,1.325079,168,0.662540


In [31]:
data_B05  ## result

,cycle,capacity,SOH
0,1,1.856487,0.928244
1,1,1.856487,0.928244
2,1,1.856487,0.928244
3,1,1.856487,0.928244
4,1,1.856487,0.928244
...,...,...,...
50280,168,1.325079,0.662540
50281,168,1.325079,0.662540
50282,168,1.325079,0.662540
50283,168,1.325079,0.662540


In [38]:
for i in range(len(listdir)):
    print("Shape of data :", np.shape(globals()['data_{}'.format(num[i])]))

Shape of data : (50285, 3)
Shape of data : (50285, 3)
Shape of data : (34866, 3)
Shape of data : (47065, 3)


KeyError: ignored

In [ ]:
print(listdir)
dataset = data_B48["SOH"]
cycle = data_B48['cycle']

dataset = np.array(dataset)
dataset = dataset.reshape((len(dataset),1))
dataset.shape

2.Split train and test dataset


In [ ]:
train_size = int(len(dataset) * 0.5)
# train_size = int(len(dataset) * 0.5)
test_size = len(dataset) - train_size
train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]
print(len(train), len(test))

In [ ]:
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset) - look_back):
        a = dataset[i:(i + look_back), 0]
        dataX.append(a)
        dataY.append(dataset[i + look_back, 0])
    print(len(dataY))
    return np.array(dataX), np.array(dataY)

In [ ]:
look_back = 1
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)

In [ ]:
trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

print(trainX.shape)
print(testX.shape)

3.Training with LSTM


In [ ]:
model = Sequential()
model.add(LSTM(64, input_shape=(trainX.shape[1], trainX.shape[2])))

model.add(Dense(1))

model.compile(loss='mae', optimizer='adam')

In [ ]:
history = model.fit(trainX, trainY, epochs=100, batch_size=20, validation_data=(testX, testY), verbose=1, shuffle = False)


In [ ]:
# save model architecture
model_json = model.to_json()
open('50%/model/B48_model.json', 'w').write(model_json)

# save model's learned weights
model.save_weights('50%/model/B48_weights.h5', overwrite=True)

In [ ]:
# Load trained model
from keras.models import model_from_json

json_file = open("50%/model/B48_model.json", "r")
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)

# model weight load
loaded_model.load_weights("50%/model/B48_weights.h5")
print("Loaded model from disk")

In [ ]:
plt.plot(history.history['loss'], label='train_efficiency')
plt.plot(history.history['val_loss'], label='test_efficiency')
plt.axis([-5, 100, 0, 0.07])
plt.legend()
plt.show()

In [ ]:
yhat = model.predict(testX)
tyhat = model.predict(trainX)
plt.plot(yhat, label='Prediction')
plt.plot(testY, label='Real data')
plt.legend()
plt.show()

In [ ]:
print("Shape of train data :", trainX.shape)
print("Shape of prediction :", yhat.shape)
print("Shape of real data :", testY.shape)

In [ ]:
rmse = math.sqrt(mean_squared_error(testY, yhat))
mae = mean_absolute_error(testY, yhat)
print('Test RMSE: %.3f' % rmse)
print('Test MAE: %.3f' % mae)

4.Visualisation

In [ ]:
print("Shape of cycle :", np.shape(cycle))

cycle1 = cycle[0:train_size-1]
cycle2 = cycle[train_size+1:len(dataset)]

# cycle1 = cycle[0:train_size]
# cycle2 = cycle[train_size:len(dataset)]

print(np.shape(cycle1), np.shape(cycle2))

In [ ]:
sns.set_style("darkgrid")
plt.figure(figsize=(12, 8))

plt.plot(cycle1, trainX[:,0],label='Used real data', linewidth = 3, color='r')
plt.plot(cycle2, testY,label='Real data', linewidth = 3, color='b')
#plt.plot(cycle2, yhat[:,0],label='LSTM Prediction', linewidth = 3, color='g')
plt.plot(cycle2, yhat[:,0],label='LSTM Prediction', linewidth = 3, color='g')
plt.legend(prop={'size': 16})

plt.ylabel('SoH', fontsize = 15)
plt.xlabel('Discharge cycle', fontsize = 15)
plt.title(str(num[7]) + " SOH Prediction", fontsize = 15)
plt.savefig('50%/fig/' + str(num[7]) + '_LSTM.jpg')
plt.show()

# Nouvelle section